# Hyperion Capstone 7

This project will focus on comparing the [K-mean](https://en.wikipedia.org/wiki/K-means_clustering) and the [KNN](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm) methods of clustering data after applying Principle Componant Analysis. The dataset used in this notebook will be [US arrest statistics](https://www.kaggle.com/datasets/kurohana/usarrets). This dataset is described as; Arrests per 100,000 residents for assult, murder, and rape in each of the 50 US states in 1973.

The project strucutre:
- A small EDA on the dataset
- Applying PCA and tuning the number of principle componants
- Training a K-mean model
- Training a KNN model
- Comparing both models

In [ ]:
#Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from itertools import combinations


## Loading and exploring data

In this section I will be looking at the data set and looking at the different attributes and how they link together. I will also be dealing with any empty values.

In [ ]:
#Load data
df = pd.read_csv("UsArrests.csv")

#Checking for empty cells
df.info()

#Gets the first 5 rows
print("\n\nTop of the dataset:\n",df.head())

This initial opening of the dataset tells us that there are no empty cells that need to be dealt with. It also seems that the attribute names "City" is named wrong and should be "State" instead. The next step will be analysing the distribution of each statistic throughout the dataset.

In [ ]:
#Changes name of the "City" column
df.rename(columns={"City":"State"},inplace=True)

#Dict to convert US state names to abbreviations
us_state_to_abbrev = {
"Alabama": "AL",
"Alaska": "AK",
"Arizona": "AZ",
"Arkansas": "AR",
"California": "CA",
"Colorado": "CO",
"Connecticut": "CT",
"Delaware": "DE",
"Florida": "FL",
"Georgia": "GA",
"Hawaii": "HI",
"Idaho": "ID",
"Illinois": "IL",
"Indiana": "IN",
"Iowa": "IA",
"Kansas": "KS",
"Kentucky": "KY",
"Louisiana": "LA",
"Maine": "ME",
"Maryland": "MD",
"Massachusetts": "MA",
"Michigan": "MI",
"Minnesota": "MN",
"Mississippi": "MS",
"Missouri": "MO",
"Montana": "MT",
"Nebraska": "NE",
"Nevada": "NV",
"New Hampshire": "NH",
"New Jersey": "NJ",
"New Mexico": "NM",
"New York": "NY",
"North Carolina": "NC",
"North Dakota": "ND",
"Ohio": "OH",
"Oklahoma": "OK",
"Oregon": "OR",
"Pennsylvania": "PA",
"Rhode Island": "RI",
"South Carolina": "SC",
"South Dakota": "SD",
"Tennessee": "TN",
"Texas": "TX",
"Utah": "UT",
"Vermont": "VT",
"Virginia": "VA",
"Washington": "WA",
"West Virginia": "WV",
"Wisconsin": "WI",
"Wyoming": "WY",
"District of Columbia": "DC",
"American Samoa": "AS",
"Guam": "GU",
"Northern Mariana Islands": "MP",
"Puerto Rico": "PR",
"United States Minor Outlying Islands": "UM",
"U.S. Virgin Islands": "VI",
}

df.State = df.State.map(us_state_to_abbrev)

#Selects subset only containing numeric data
df_num = df.drop("State",axis=1).set_index(df.State.values)

#Generates subplots
fig,axs = plt.subplots(1,4,figsize=(20,5))

#Plots a histogram of each numeric column
for col,ax in zip(df_num,fig.axes):
    df[col].plot.hist(ax=ax)
    ax.set_xlabel(col)

**Murder per Capita**
- The most common rate of murder is around 2.5 and 7.5.
- Rates above 11 are much rarer.

**Assult per Capita**
- The rates have a similar shape to the murder rates, but the absolute values are much higher.
- Peak at 125, though there is another peak at 275.

**Urban Population**
- Most states have an urban population of around 65%.
- There seems to be a much more even spread of populations between 45% and 95% compared to the other statistic types.

**Rape per Capita**
- More rape occures in general than murders
- More assults then rapes
- There a rapid dropoff for rates above 25


The next step will be assessing if the dataset is a good fit for the application of PCA. A dataset that is a good candidate for PCA is one where there is strong positive or negative correlations between multiple columns. Meaning that linking these columns together could make the dataset easier to visualise (since there will be less than 4 columns), while not also losing a substantial amount of information.

In [ ]:
#Gets the correlation between each attribute with every other attribute
num_corr = df_num.corr()

#Plots correlation grid
sns.heatmap(num_corr,annot=True,vmin=-1,vmax=1,cmap="RdBu").set(title="Correlation Grid")

**Observations**
- None of the attributes shows a negative correlation.
- Most show a strong positive correlation.
- Murder rates and Urban population show the least correlation with a value of 0.07
- Assault rate and Urban population also shows a weak positive correlation
- Murder and assault rates have the highest correlation of 0.8

Overall, most of the attributes shows some sort of correlation with another. Therefore, doing a PCA would be beneficial and may result in data that could be visualised. However, shown in the histograms above, the range of values for each column is very different. For instance, the Urban population is given in percentage, while all other columns are per 100,000. Even within the columns that are per 100,000, there is a difference in their range. For example, murder rates reach a high of below 20, while assault rates reach a high of above 300.

This big difference in the columns will cause the PCA to favour the larger values which may skew the end result. For this reason, I will now normalise the data.

In [ ]:
#Scale numric data and put into a new dataframe
df_num_scaled = pd.DataFrame(StandardScaler().fit_transform(df_num),columns=df_num.columns,index=df_num.index)

#Shows that the data has been scaled
print(f"{'Unscaled data':=^50}")
print(df_num.describe())
print(f"\n{'Scaled data':=^50}")
print(df_num_scaled.describe())

#Plots histogram of both datasets to verify scaling
fig,axs = plt.subplots(2,2)
fig.tight_layout()

#Murder plots
df_num.Murder.plot.hist(ax=axs[0,0])
axs[0,0].set_title("Unscaled Murder")
df_num_scaled.Murder.plot.hist(ax=axs[1,0])
axs[1,0].set_title("Scaled Murder")

#Assault plots
df_num.Assault.plot.hist(ax=axs[0,1])
axs[0,1].set_title("Unscaled Assault")
df_num_scaled.Assault.plot.hist(ax=axs[1,1])
axs[1,1].set_title("Scaled Assault")

After scaling the dataset, we can now see that all columns have a mean of 0, and a standard deviation of 1. This is futher shown in the histograms showing the unscaled and scaled data. Now both murder and assault have values that lie arounf -2 and 2, but the overall shape of the plots are conserved.

Before carrying out the PCA, a little more exploration can be done on the scaled dataset.

In [ ]:
#Gets the state with the highest and lowest statistic for each column
for col in df_num:
    top = df_num[col].idxmax()
    bot = df_num[col].idxmin()
    print(f"{col}:")
    print(f"Top: {top} | Bottom: {bot}\n")

#Plots the top 5 states for each statistic
fig,axs = plt.subplots(2,4,figsize=(20,5))
fig.tight_layout(pad=5.5)
for i,col in enumerate(df_num):
    top_5 = df_num[col].sort_values(ascending=False).iloc[:5]
    top_5.plot.bar(ax=axs[0,i],color="r")
    axs[0,i].set_title(col)
    axs[0,i].tick_params(axis='x', labelrotation=45)

    bot_5 = df_num[col].sort_values(ascending=True).iloc[:5]
    bot_5.plot.bar(ax=axs[1,i],color="g")
    axs[1,i].tick_params(axis='x', labelrotation=45)

axs[0,0].set_ylabel("Top 5")
axs[1,0].set_ylabel("Bottom 5")


After looking at the extremes of the dataset, we can say that North Dakota is one of the safer states, while also being one with the lowest urban population. The same can be said for Vermont. Overall, a lot of the bottom 5 states in each statistic come up multiple times in other statistics.

On the other hand, this is not the case for the top 5. For instance, Florida only shows in in the Murder and Assault stats.

It is interesting to note that Mississippi has one of the highest murder rates, but also one of the lowest percentage of urban population.

## PCA

This section will be dedicated to the PCA of the data. A comparison to the scaled and unscaled data will be shown to demonstrate the importance of scaling the data. After that, the ratio of explained variance will be used to determine the optimal number of columns needed to reduce the complexity of the dataset, while also losing the least amount of information.

Once this analysis is done and the optimal number of columns if found, a clustering model will be applied to the data.

The first step is to apply the PCA to the datasets and visualise it. The visualisation will be done using a biplot. The biplot will show the position of the datapoints relative to some selected principle axes given by the PCA. It will also show the direction and importance of the original axes on relation to those axes.

In [ ]:
def biplot(score,coeff,labels=None,points=None):
    """Plots a biplot that shows specific principle axes along with the datapoints"""

    #Gets data points from pca transformed data
    xs = score[:,0]
    ys = score[:,1]

    #Gets number of prinicple axes
    n = coeff.shape[0]

    #Scaled datapoints
    scalex = 1.0/(xs.max() - xs.min())
    scaley = 1.0/(ys.max() - ys.min())

    fig, ax = plt.subplots()    

    #Plots the dataponits wrt principle axes
    ax.scatter(xs * scalex,ys * scaley,s=5)

    #Labels the datapoints
    for i in range(0,len(xs)):
        txt = points[i]
        ax.annotate(txt, (xs[i]* scalex, ys[i]* scaley))

    #Draws oringinal axes wrt to new principle axes
    for i in range(n):
        ax.arrow(0, 0, coeff[i,0], coeff[i,1],color = 'r',alpha = 0.5)
        if labels is None:
            ax.text(coeff[i,0]* 1.15, coeff[i,1] * 1.15, "Var"+str(i+1), color = 'green', ha = 'center', va = 'center')
        else:
            ax.text(coeff[i,0]* 1.15, coeff[i,1] * 1.15, labels[i], color = 'g', ha = 'center', va = 'center')
 
    plt.grid()

    # Adapted from: https://ostwalprasad.github.io/machine-learning/PCA-using-python.html

In [ ]:
#Generates PCA object
r = 10
pca = PCA(random_state=r)

#Applies PCA to unscaled data
pca_unscaled_array = pca.fit_transform(df_num)
pca_unscaled = pd.DataFrame(pca_unscaled_array,index=df_num.index)

#Gets the principle axes components
unscaled_comps = pca.components_

#Plots biplot
biplot(pca_unscaled_array[:,0:2],np.transpose(unscaled_comps[0:2,:]),df_num.columns,df_num.index)
plt.xlabel("PC1")
plt.ylabel("PC2")


This biplot of the unscaled data shows that the Urban population and Assault columns dominate the principle axes, this is shown by the length of the red arrows. This makes sense since compared to the other columns, their range of values is much larger. We ideally want all the oringial axes to have a similar influence on the principle axes. This can be fixed by scaling the data. 

In [ ]:
#Transforms scaled data via PCA
pca = PCA()
pca_scaled_array = pca.fit_transform(df_num_scaled)
pca_scaled = pd.DataFrame(pca_scaled_array,index=df_num_scaled.index)

#Gets principle axes components
scaled_comps = pca.components_

#Plots biplot using PC1,PC2
biplot(pca_scaled_array[:,0:2],np.transpose(scaled_comps[0:2,:]),df_num_scaled.columns,df_num_scaled.index)
plt.xlabel("PC1")
plt.ylabel("PC2")


In [ ]:
#Plots biplot using PC1,PC2
biplot(pca_scaled_array[:,[2,0]],np.transpose(scaled_comps[[2,0],:]),df_num_scaled.columns,df_num_scaled.index)
plt.xlabel("PC3")
plt.ylabel("PC1")

In [ ]:
biplot(pca_scaled_array[:,[1,2]],np.transpose(scaled_comps[[1,2],:]),df_num_scaled.columns,df_num_scaled.index)
plt.xlabel("PC2")
plt.ylabel("PC3")

Now that we have scaled the data, the biplot is much more even. Each of the original axes now have similar lengths and therefore similar importance levels.

The biplot indicates that the first principle axes (PC1) represents some sort of general danger level of a state, since all 3 columns that represent some crime are all points along the PC1 axes. The 2nd principle axes seems to represent the general population distribution of the state, having states with larger urban populations have negative PC2 values.

The next stage of the analysis to to determine the optimal number of columns. This is done by looking at how the explined variance changes depending on the number of principle axes used.

In [ ]:
#Plots the cumulative explained variance and the abs explained varaince
fig, axs = plt.subplots(2,sharex='col')

axs[0].plot(np.cumsum(pca.explained_variance_ratio_))
axs[0].set_ylabel("Cumulative explained variance")
axs[1].plot(pca.explained_variance_ratio_)
axs[1].set_ylabel("Explained variance")
axs[1].set_xlabel("Number of principle axes")


These plots show that the increase in explained variance is very low after the 3rd principle axis has been generated. However, the amount of explained variance is reduced significantly if we only have 2 principle axes or lower. This leads me to think that have 3 is the best choice. 

Another benefit of only having 3 principle axis is that a 3D plot can be visualised where the clustering may be more straightforward to understand once that has been completed.

In [ ]:
#Removes the unwanted principle axis
final_pca = pca_scaled.drop(3,axis=1)

final_pca.head(10)

## Clustering - K-Mean

In this part of the report, I will be using a k-mean model of unsupervised learning to try and organise my data into clusters.

The first step will be to find the number of clusters that are optimum. This can be done by applying k-means to the dataset specifying a different number of clusters each time. The [silhouette score](https://en.wikipedia.org/wiki/Silhouette_(clustering)) can then be found and plotted for each test. 

The silhouette score is a metric that is used to measure if the predicted clusters are well distinguished from eachother or too close together. A good clustering outcome would be one where the clusters are separated enough to be distinct from eachother. The score itself can be between -1 and 1.

- 1: Clusters are well spread out
- 0: Clusters are overlapping completely
- -1: Clusters are predicted wrong

In [ ]:
print("Silhouette Scores:")

#Applys K-mean to the data using a range of cluster options
score = []
for i in range(2,10):
    model = KMeans(n_clusters=i,random_state=r)
    model.fit(final_pca)
    score.append(silhouette_score(final_pca,model.labels_))

#Plots all the silhouette scores collected
plt.plot(list(range(2,10)),score)
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Score")

The silhouette scores indicate that the best number of clusters should be 2. This is shown due to the fact that the score very quickly decreases if there are more than 2 clusters. Therefore, the clusters are becoming more and more similar, which is not what we want when we want clusters. 

Now we will run the clustering again but with only 2 clusters and visualise the data.

In [ ]:
#Trains and predicts using k-means to find 2 clusters
kmean = KMeans(n_clusters=2,random_state=r)
cluster_labels = [int(value) for value in kmean.fit_predict(final_pca)]

#Separates data into 3 variables and the labels
x = final_pca[0]
y = final_pca[1]
z = final_pca[2]
labels = final_pca.index

#Gets combinations of 3 principle axes
combo = combinations(final_pca,2)

#Plots the datapoints with thier relatvent clustering
fig, axs = plt.subplots(1,3,figsize=(15,5))
for axis,comp in zip(fig.axes,combo):
    
    #Gets the principle axis 
    comp = list(comp)
    xs = comp[0]
    ys = comp[1]

    #Plots the scatter
    final_pca[comp].plot.scatter(x=xs,y=ys,ax=axis,color=cluster_labels,cmap=matplotlib.colors.ListedColormap(["red","blue"]))
    axis.set_xlabel(f"PC{comp[0]+1}")
    axis.set_ylabel(f"PC{comp[1]+1}")

    #Label each point in the plot
    for state in final_pca.index:
        axis.annotate(state,(final_pca.loc[state,xs],final_pca.loc[state,ys]))



In [ ]:

#Generates 3D plot
fig = plt.figure()
ax = fig.add_subplot(projection="3d")

#plots the scatter using the clustering results for colouring points
ax.scatter(x,y,z,c=cluster_labels,cmap="RdBu")
ax.set_xlabel("PC1")
ax.set_ylabel("PC2")
ax.set_zlabel("PC3")
ax.view_init(20,85)


In [ ]:
#Initaise dataframe that will contain the importance of each original axis with each new principle axis
importance = pd.DataFrame({"Attr":df_num.columns})

#Goes through each 
for i,comp in enumerate(pca.components_):
    importance[f"PC{i+1}"] = abs(comp)

importance.drop("PC4",axis=1,inplace=True)
importance

After plotting the points with respect to all of the combinations of the principle axes, the 2 groups of countries can easily be seen when looking at the PC1-PC2 and the PC1-PC3 plots. This relationship can be futher visualised using the 3D plot above. 

When trying to distinguish what the clusters represent, we need to think about the underlying priniciple axes as done above.

PC1: A general danger level of the state, since it is strongly related to the murder, assault, and rape statistics, while also being related to urban population, but not by much.

PC2: Tried to separate more rural states from more urban states but also taking into account their murder rates. So a more urban state with a high murder rate could land somewhere close to a rural state with a low murder rate.

PC3: Tries to separate the types of crimes taking place, but favours the rape statistic more than others. 

In [ ]:
crime = ["Murder","Assault","Rape"]
print("SHOWING PC1 TRENDS - Sum of all stats")
print("(TX):",df_num_scaled.loc["TX"].sum())
print("(KS):",df_num_scaled.loc["KS"].sum())
print(f"{'':=^50}")

print("SHOWING PC2 TRENDS")
print("(CO):",dict(df_num_scaled.loc["CO",["UrbanPop","Murder"]]))
print("(NV):",dict(df_num_scaled.loc["NV",["UrbanPop","Murder"]]))
print(f"{'':=^50}")

print("SHOWING PC3 TRENDS - Rape stats")
print("(OR):",df_num_scaled.loc["OR","Rape"])
print("(WA):",df_num_scaled.loc["WA","Rape"])
print("(DE):",df_num_scaled.loc["DE","Rape"])



The trends have been shown by picking certain states that have some sort of relationship to eachother. For example, to show the PC1 is the general risk level of a state, picking TX and KS, which are on opposite ends of the plot shows that TX has much more crime and a higher urban population than KS. which is what was expected.

To varify PC2, CO and NV have been picked since they are close to eachother, however, even though NV has a higher urban population, the murder rate means that is is placed slightly more positive on the PC2 axis than CO. Which is what was expected.

The varify PC3, the rape statisics of a few states were chosen. They were chosen because each state was slightly more negavtive than the last on the PC3 axis, Therefore, the rape stat reduced. Which was expected.

Knowing this as looking at the 3D scatter plot, I can say that the main separation between the 2 clusters was along the PC2 axis, meaning that PC1 was the main factor in deciding which cluster a point was put into. Indicating that the clusters are sorted into states that are generally more dangerous and states that are generally safer.

## Clustering - KNN